In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date
from datetime import datetime

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor, Ridge, Lasso
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from tensorflow.keras import models, layers 

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

sns.set_style(style=("darkgrid"))
plt.figure(figsize=(10, 8))

In [ ]:
df = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

The data look's clean so i moved staright to EDA.

In [ ]:
df_train = df.copy()

In [ ]:
df_train["revenue"].hist()

In [ ]:
df_train["cat_revenue"] = pd.cut(df_train["revenue"],
                               bins=[0, 5000000, 10000000, np.inf],
                               labels=["low", "mid", "high"])
df_train.head()

Convert Date

In [ ]:
def transform_date(dataframe):
    today = date.today()
    days = []
    for value in dataframe["Open Date"].values:
        casting_date = datetime.strptime(value, "%m/%d/%Y").date()
        delta = today - casting_date
        day = delta.days
        days.append(day)

    dataframe["Open Date"] = pd.Series(days, name="Open Days")
    return dataframe

In [ ]:
df_train = transform_date(df_train)
df_train.head()

In [ ]:
df_train["Open Date"].hist()

In [ ]:
df_train["cat_age"] = pd.cut(df_train["Open Date"],
                           bins=[0, 4000, 7000, np.inf],
                           labels=['new', "mid", "old"])
df_train.head()

Seprate Categorical Features

In [ ]:
df_vis = df_train.select_dtypes(exclude=("int64", "float64"))
df_vis.head()

In [ ]:
sns.countplot(data=df_vis, x="City Group", palette="flare")

In [ ]:
sns.countplot(data=df_vis, x="City", palette="flare")

In [ ]:
sns.countplot(data=df_vis, x="Type", palette="flare")

In [ ]:
sns.countplot(data=df_vis, x="cat_revenue", palette="flare")

In [ ]:
sns.countplot(data=df_vis, x="cat_age", palette="flare")

In [ ]:
sns.catplot(data=df_vis, x="City Group", hue="cat_revenue", col="Type", kind="count", palette="flare")

In [ ]:
sns.catplot(data=df_vis, x="City Group", hue="cat_revenue", col="cat_age", kind="count", palette="flare")

In [ ]:
sns.catplot(data=df_vis, x="Type", hue="cat_revenue", col="City Group", kind="count", palette="flare")

In [ ]:
sns.catplot(data=df_vis, x="Type", hue="cat_age", col="City Group", kind="count", palette="flare")

In [ ]:
sns.catplot(data=df_vis, x="Type", hue="City Group", col="cat_age", kind="count", palette="flare")

In [ ]:
sns.catplot(data=df_vis, x="cat_revenue", hue="cat_age", col="Type", palette="flare", kind="count")

In [ ]:
df_train.head()

In [ ]:
X_train = df_train.drop(["Id", "City", "revenue", "cat_revenue", "cat_age"], axis=1)
y_train = df_train["revenue"]

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_train.info()

In [ ]:
X_train_num = X_train.select_dtypes(include=("int64", "float64"))
X_train_num.head()

In [ ]:
X_train_cat = X_train.select_dtypes(include=("object"))
X_train_cat.head()

In [ ]:
num_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")),
                         ("scaler", StandardScaler())])

In [ ]:
num_features_name = list(X_train_num.columns)
cat_ordinal_name = ["City Group"]
cat_onehot_name = ["Type"]

full_pipeline = ColumnTransformer([("num features", num_pipeline, num_features_name),
                                   ("ordinal encoding", OrdinalEncoder(), cat_ordinal_name),
                                   ("one hot encoding", OneHotEncoder(), cat_onehot_name)])

X_train_preprocessed = full_pipeline.fit_transform(X_train)

In [ ]:
dataframe = pd.DataFrame(X_train_preprocessed)
dataframe.head()

In [ ]:
pca = PCA(n_components=0.99)
X_train_pca = pca.fit_transform(dataframe)
pca.n_components_

In [ ]:
def modeling(stimator, X_train, y_train, cv=3, scoring="neg_mean_squared_error"):
    
    model = stimator
    model.fit(X_train, y_train)
    pred = model.predict(X_train)
    mse = mean_squared_error(y_train, pred)
    rmse = np.sqrt(mean_squared_error(y_train, pred))
    score = r2_score(y_train, pred)
    
    print(stimator)
    print(f"mse on training data: {mse}")
    print(f"rmse on training data: {rmse}")
    print(f"r2_score: {score}")
    scores = cross_val_score(stimator, X_train, y_train, cv=cv, scoring=scoring)
    
    if scoring=="neg_mean_squared_error":
        mean_mse = np.mean(-sum(scores))
        mean_rmse = np.sqrt(mean_mse)
        print(f"mean mse on cross val data: {mean_mse}")
        print(f"mean rmse on cross val data: {mean_rmse}")
    
    elif scoring=="r2":
        mean_score = np.mean(scores)
        print(f"mean r2 score on cross val data: {mean_score}")

    print("__________________________________________________________________________________________________")

In [ ]:
regressors = {"SVR": SVR(),
              "RandomForestRegressor": RandomForestRegressor(),
              "AdaBoostRegressor": AdaBoostRegressor(),
              "KNN": KNeighborsRegressor(),
              "SGD": SGDRegressor(), 
              "Ridge": Ridge(),
              "Lasso": Lasso()}

In [ ]:
for regressor in regressors.values():
    modeling(regressor, X_train_pca, y_train)

In [ ]:
num_pipeline2 = Pipeline([("imputer", SimpleImputer(strategy="median")),
                         ("scaler", MinMaxScaler())])

num_features_name = list(X_train_num.columns)
cat_ordinal_name = ["City Group"]
cat_onehot_name = ["Type"]

full_pipeline_nn = ColumnTransformer([("num features", num_pipeline2, num_features_name),
                                   ("ordinal encoding", OrdinalEncoder(), cat_ordinal_name),
                                   ("one hot encoding", OneHotEncoder(), cat_onehot_name)])

X_train_preprocessed_nn = full_pipeline_nn.fit_transform(X_train)

In [ ]:
dataframe2 = pd.DataFrame(X_train_preprocessed_nn)

pca_nn = PCA(n_components=0.95)
X_train_nn_pca = pca_nn.fit_transform(dataframe2)
pca_nn.n_components_

In [ ]:
net = models.Sequential([layers.Dense(200, "relu"),
                        layers.Dropout(0.5),
                        layers.Dense(100, "relu"),
                        layers.Dropout(0.5),
                        layers.Dense(100, "relu"),
                        layers.Dropout(0.5),
                        layers.Dense(1)])
net.compile(optimizer="adam",
            loss="mse")
H2 = net.fit(X_train_nn_pca, y_train, validation_split=0.3, batch_size=20, epochs=200)

In [ ]:
pd.DataFrame(H2.history).plot(figsize=(10, 8))
plt.gca()
plt.show()
